<a href="https://colab.research.google.com/github/pramodith/llm_exploration/blob/colab/dynamic_prompt_token_dropping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt pruning via Dynamic Token Dropout

In [11]:
!pip install transformers
!pip install langchain
!pip install datasets
!pip install huggingface_hub
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [102]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Dict, Any, Optional, Union, Tuple
import torch
from pprint import pprint
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datasets import load_dataset


ImportError: cannot import name 'get_session' from 'huggingface_hub.utils' (c:\Users\pramo\anaconda3\envs\PromptRiddler\lib\site-packages\huggingface_hub\utils\__init__.py)

### Pick a Student Model
The model needs to be generative!

In [88]:
def get_model_and_tokenizer(model_name: str) -> Tuple[AutoModelForCausalLM, AutoTokenizer]: 
    """
    This function returns a model and tokenizer from a given model name.

    Args:
        model_name (str): The name of the model to be loaded.

    Returns:
        AutoModelForCausalLM, AutoTokenizer: The model and tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.eval()
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id
    return model, tokenizer


### Get individual token probabilities
Our key idea is that any token that can be predicted with a high probability by a smaller student model like `gpt2` can also be
easily predicted by an LLM, and as a result can be omitted from our input documents. This way we can preserve the key information
in our prompt, without losing out on performance.

The first step to being able to drop the input tokens with high probability is to obtain the token probabilities of all tokens in 
our prompt using a student model, which we do via the `to_tokens_and_probs` function.


The function `to_tokens_and_probs` takes a list of input texts and a language model, and returns a list of tuples for each token in the input text. Each tuple contains the decoded token, the probability of that token being generated, the position of the token in the input, and the token's ID.

Here's a step-by-step breakdown of what the function does:

1. It tokenizes the input texts and feeds them into the model.
2. It applies a softmax function to the logits output by the model to get the probabilities of each token.
3. It shifts the probabilities and input IDs to align with the generated tokens.
4. It collects the probability of each generated token.
5. It iterates over each token and its corresponding probability. If the token is not a special token, it appends a tuple containing the decoded token, the probability, the token position, and the token ID to the list for that sentence.
6. It returns a list of these lists, one for each input sentence.

This function is useful for understanding the probabilities that a language model assigns to each token when generating text.

In [96]:
def to_tokens_and_probs(model: AutoModelForCausalLM, tokenizer: AutoTokenizer, input_texts: List[str]) -> Tuple[str, float, int, int]:
    """
    This function takes a list of input texts and returns a list of tuples (decoded_token, prob, token_position, token) 
    for each token in the input text.
    Reference: https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075/17

    Args:
        model (AutoModelForCausalLM): The model to be used.
        tokenizer (AutoTokenizer): The tokenizer to be used.
        input_texts (List[str]): The input texts to be used.

    Returns:
        Tuple[str, float, int, int]: A list of tuples (decoded_token, prob, token_position, token) for each token in the input text.
    """
    input_ids = tokenizer(input_texts, padding=True, return_tensors="pt").input_ids
    outputs = model(input_ids)
    probs = torch.softmax(outputs.logits, dim=-1).detach()

    # collect the probability of the generated token -- probability at index 0 corresponds to the token at index 1
    probs = probs[:, :-1, :]
    input_ids = input_ids[:, 1:]
    gen_probs = torch.gather(probs, 2, input_ids[:, :, None]).squeeze(-1)

    batch = []
    for input_sentence, input_probs in zip(input_ids, gen_probs):
        text_sequence = []
        for token_pos, (token, p) in enumerate(zip(input_sentence, input_probs)):
            if token not in tokenizer.all_special_ids:
                text_sequence.append((tokenizer.decode(token), p.item(), token_pos, token.item()))
        batch.append(text_sequence)
    return batch

In [93]:
sample_prompt = ["The capital of France is Paris."]
model, tokenizer = get_model_and_tokenizer("facebook/opt-125m")
token_probs = to_tokens_and_probs(model, tokenizer, sample_prompt)


In [94]:
token_probs

[[('The', 0.08746550977230072, 0, 133),
  (' capital', 8.743480429984629e-05, 1, 812),
  (' of', 0.14376778900623322, 2, 9),
  (' France', 0.0054758754558861256, 3, 1470),
  (' is', 0.24484363198280334, 4, 16),
  (' Paris', 0.006779637187719345, 5, 2201),
  ('.', 0.3360891044139862, 6, 4)]]

### Create a Langchain Chat Client
Set your key to openai's API's using colab's secrets features.

In [95]:
try:
    chat = ChatOpenAI(temperature=0)
except Exception as e:
    print(e)
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    chat = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [97]:
# Cost per 1k input tokens as of 11/15/2023
GPT_35_TURBO_COST_PER_INP_TOKEN = 0.0010/1000

### Prompt Definition
We'll define our prompts system and human messages inside a Pydantic model. We'll evaluate the LLM on SQUAD, a reading comprehension dataset. Each record in the dataset contains a passage, a question based on the passage, and the answer.
In our experimetns we'll feed the llm with the passage and question as our baseline. Then we'll feed the llm with the passage and question after dropping tokens based on our student model.

We'll then compare the cost and accuracies of the two experiments. Ideally, we'd like the LLMs performance to be the same with and without dropped tokens, since this ensures that we don't have to trade any 💵 saved for decreased performance.

In [99]:
from pydantic import BaseModel
class ReadingComprehensionPrompt(BaseModel):
    system_message: str = SystemMessagePromptTemplate.from_template(f"You are a very smart student in a reading comprehension class."
        "Your teacher is giving you a reading comprehension test. You are given a passage and a question."
        "You must answer the question based on the passage. Be as concise as possible, correct any typos, you may return the excerpt from the passage containing the answer without phrasing it as a sentence.")
    human_message: str = HumanMessagePromptTemplate.from_template("Passage: {passage}\nQuestion: {question}\nAnswer: ")

rc_prompt = ReadingComprehensionPrompt()

# LLM Response and Cost Per Query
The function `get_llm_response` is designed to generate a response from a language model for a given passage and question. It also
returns an openai callback object that returns details like the input/output token count, total cost etc.

Here's a detailed breakdown of its purpose and functionality:

- **Parameters:**
  - `prompt`: An instance of the `BaseModel` class, which provides the system and human messages for the chat prompt.
  - `chat`: An instance of the `ChatOpenAI` class, which is used to interact with the language model.
  - `passage`: A string representing the passage to be read.
  - `question`: A string representing the question to be answered.

- **Functionality:**
  - The function first creates a `ChatPromptTemplate` from the system and human messages in the prompt.
  - It then formats the prompt with the given passage and question, and converts it to a list of messages.
  - It sets up a callback to the OpenAI API and uses the `chat` instance to generate a response from the language model.
  - Finally, it returns the response and the callback.

- **Returns:**
  - `response`: The response generated by the language model.
  - `cb`: The callback to the OpenAI API.

This function is useful for generating language model responses for a reading comprehension task.

In [100]:
def get_llm_response(prompt: BaseModel, chat: ChatOpenAI, passage: str, question: str):
    chat_message = ChatPromptTemplate.from_messages([prompt.system_message, prompt.human_message])
    completed_prompt = chat_message.format_prompt(passage=passage, question=question).to_messages()
    with get_openai_callback() as cb:
      response = chat(completed_prompt)
    return response, cb

In [103]:
get_llm_response(rc_prompt, chat, "The capital of France is Paris.", "What is the capital of Australia?")

(AIMessage(content='The passage does not provide information about the capital of Australia.', additional_kwargs={}, example=False),
 Tokens Used: 115
 	Prompt Tokens: 103
 	Completion Tokens: 12
 Successful Requests: 1
 Total Cost (USD): $0.0001785)

### Downloading the Squad Dataset

In [104]:
def sample_squad_dataset(num_samples: int = 100):
    dataset = load_dataset("squad", split="validation")
    return dataset.shuffle().select(range(num_samples))

In [105]:
squad_dataset = sample_squad_dataset()

NameError: name 'load_dataset' is not defined

In [19]:
squad_dataset[0]

{'id': '5729f9953f37b31900478620',
 'title': 'Immune_system',
 'context': 'Hormones can act as immunomodulators, altering the sensitivity of the immune system. For example, female sex hormones are known immunostimulators of both adaptive and innate immune responses. Some autoimmune diseases such as lupus erythematosus strike women preferentially, and their onset often coincides with puberty. By contrast, male sex hormones such as testosterone seem to be immunosuppressive. Other hormones appear to regulate the immune system as well, most notably prolactin, growth hormone and vitamin D.',
 'question': 'Female sex hormones are immunostimulators of which immune responses?',
 'answers': {'text': ['adaptive and innate immune responses',
   'both adaptive and innate',
   'adaptive and innate immune responses'],
  'answer_start': [154, 149, 154]}}

### Dropping tokens in the passage and question
The function `get_token_dropped_text` is designed to drop a certain percentage of tokens from a given text based on the token probabilities generated by the chosen student language model.

Here's a detailed breakdown of its purpose and functionality:

- **Parameters:**
  - `doc`: A string representing the document from which tokens will be dropped.
  - `tokenizer`: An instance of the `AutoTokenizer` class, which is used to tokenize the document.
  - `dropout_percent`: A float representing the percentage of tokens to be dropped from the document.

- **Functionality:**
  - The function first uses the `to_tokens_and_probs` function to get the probabilities of each token in the document.
  - It then sorts these tokens based on their probabilities and selects the top `dropout_percent` of tokens to be dropped.
  - It removes these tokens from the document and decodes the remaining tokens back into text using the tokenizer.

- **Returns:**
  - `dropped_token_text`: The text after the top `dropout_percent` of tokens have been dropped.
  - `dropped_tokens`: A list of the tokens that were dropped.

In [106]:
import numpy as np
def get_token_dropped_text(doc: str, tokenizer:AutoTokenizer, dropout_percent=0.1) -> str:
    token_probs = to_tokens_and_probs(model, tokenizer, [doc])
    tokens = [token[-1] for token in token_probs[0]]
    num_tokens_to_drop = int(len(tokens)*dropout_percent)
    top_10_percent_tokens = sorted(token_probs[0], key=lambda x: x[1], reverse=True)[:len(tokens) // num_tokens_to_drop]
    dropped_tokens = [t[0] for t in top_10_percent_tokens]
    tokens_after_deletion = np.delete(tokens, [token[2] for token in top_10_percent_tokens])
    # Remove the top 10% of tokens
    dropped_token_text = tokenizer.decode(tokens_after_deletion)
    return dropped_token_text, dropped_tokens


In [48]:
total_dropped_cost = 0
total_cost = 0
dropped_tokens = []
predicted_answer = []
predicted_answer_for_dropped = []
for i in range(len(squad_dataset)):
  dropped_context, dropped_context_text = get_token_dropped_text(squad_dataset[i]["context"], tokenizer)
  dropped_question, dropped_question_text = get_token_dropped_text(squad_dataset[i]["question"], tokenizer)
  answer, cb = get_llm_response(rc_prompt, chat, passage = squad_dataset[i]["context"], question = squad_dataset[i]["question"])
  dropped_answer, cb_d = get_llm_response(rc_prompt, chat, passage=dropped_context, question=dropped_question)
  dropped_cost = cb_d.prompt_tokens*GPT_35_TURBO_COST_PER_INP_TOKEN
  cost = cb.prompt_tokens*GPT_35_TURBO_COST_PER_INP_TOKEN
  total_dropped_cost += dropped_cost
  total_cost += cost
  print(f"Actual answer is {squad_dataset[i]['answers']}")
  print(f"Answer for dropped text is {dropped_answer}, cost is {total_dropped_cost}")
  print(f"Answer for original text is {answer}, cost is {total_cost}")
  predicted_answer.append(answer.content)
  predicted_answer_for_dropped.append(dropped_answer.content)
  dropped_tokens.extend(dropped_context_text)
  dropped_tokens.extend(dropped_question_text)

Actual answer is {'text': ['through the wilderness of the Maine district and down the Chaudière River to attack the city of Quebec', 'wilderness of the Maine district and down the Chaudière River', 'the wilderness of the Maine district', 'Maine', 'the wilderness of the Maine district and down the Chaudière River'], 'answer_start': [347, 359, 355, 377, 355]}
Answer for dropped text is content='Yes, Shirley was planning an expedition.', cost is 0.00020899999999999998
Answer for original text is content='Shirley was planning an expedition through the wilderness of the Maine district and down the Chaudière River to attack the city of Quebec.', cost is 0.00022099999999999998
Actual answer is {'text': ['William Farel', 'William Farel', 'William Farel'], 'answer_start': [519, 519, 519]}
Answer for dropped text is content='William Farel', cost is 0.000557
Answer for original text is content='William Farel', cost is 0.000592
Actual answer is {'text': ['Anheuser-Busch InBev', 'Anheuser-Busch InB

In [42]:
from collections import Counter
Counter(dropped_tokens)

Counter({'ing': 1,
         'ne': 2,
         'ac': 1,
         ',': 4,
         ' of': 10,
         ' on': 2,
         ' to': 4,
         ' for': 2,
         'ev': 3,
         'ef': 2,
         're': 4,
         ' as': 2,
         'formation': 1,
         ' needed': 1,
         'en': 1,
         'itation': 1,
         ']': 1,
         'ism': 1,
         '.': 2,
         'ots': 1,
         'u': 1,
         ' and': 1,
         ' Testament': 1,
         'ch': 1,
         'user': 1,
         'Bus': 1,
         'os': 2,
         'B': 1,
         ' own': 1,
         'it': 1,
         ' Company': 1,
         ' Bowl': 4,
         ' game': 1,
         ' year': 1,
         '-': 1,
         'rom': 1,
         'bie': 1,
         'land': 1,
         ' with': 1,
         ' chief': 1,
         ' at': 1,
         ' by': 1,
         'itions': 2})

In [43]:
squad_dataset.to_csv("llm_responses")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

4937

In [44]:
squad_dataset.add_column("predicted_answer",predicted_answer)

ArrowInvalid: ignored

In [45]:
predicted_answer

[AIMessage(content='Shirley was planning an expedition through the wilderness of the Maine district and down the Chaudière River to attack the city of Quebec.'),
 AIMessage(content='William Farel'),
 AIMessage(content='Anheuser-Busch InBev'),
 AIMessage(content='Two of the expeditions were successful.')]

### Compute Metrics
This function calculates the accuracy of the model's predictions on a given dataset.

Here's a detailed breakdown of its functionality:

- **Parameters:**
  - `ds`: This is the dataset on which the accuracy is to be calculated. Each record in this dataset is expected to have a "predicted_answer" field containing the model's prediction, and an "answers" field containing the correct answers.
  - `is_dropped_accuracy`: This is a boolean flag that determines whether the function should calculate the accuracy for the predictions made on the original text (`False`) or the text with dropped tokens (`True`).

- **Functionality:**
  - The function initializes two counters, `correct` and `wrong`, to zero.
  - It then iterates over each record in the dataset. For each record, it retrieves the list of valid answers and the predicted answer (either for the original or the dropped text, depending on `is_dropped_accuracy`).
  - It checks if the predicted answer is in the list of valid answers (ignoring case). If it is, it increments the `correct` counter and moves on to the next record. If it's not, it increments the `wrong` counter.
  - After going through all the records, it calculates the accuracy as the number of correct predictions divided by the total number of predictions (i.e., `correct / (correct + wrong)`).

- **Returns:**
  - The function returns the calculated accuracy as a float.

This function is useful for evaluating the performance of a model on a given dataset.

In [ ]:
def get_accuracy(ds, is_dropped_accuracy = False) -> float:
  """
  This function calculates the accuracy of the model on a given dataset.

  Args:
      ds (Dataset): The dataset to be used.
      is_dropped_accuracy (bool, optional): Whether to calculate the accuracy for the dropped text. Defaults to False.
  Returns:
      float: The accuracy of the model on the given dataset.
  """
  correct = wrong = 0
  for record in ds:
    valid_answers = record["answers"]["text"]
    if is_dropped_accuracy:
      predicted_answer = record["predicted_answer_for_dropped"]
    else:
      predicted_answer = record["predicted_answer"]
    found = False
    for valid_answer in valid_answers:
      if valid_answer.lower() in predicted_answer.lower():
        correct += 1
        found = True
        break
    if not found:
      print(predicted_answer, valid_answers)
      print()
      wrong += 1
  return correct/(correct + wrong)
    